#### Выгрузка данных 
Мы выгружаем файлики из Запрос ОП, ВТЦ. Там у нас лежат выгрузки по всем точкам,  
Объединяем их в один файлик.  
Также, импортируем склады и объединяем их с нашим основным документом


In [1]:
import pandas as pd
import glob
import os
from datetime import datetime, timedelta
import charset_normalizer
from datetime import time


path = (r'C:\Users\Home\Desktop\Data\Точки_продаж')

file_encoding = r'C:\Users\Home\Desktop\Data\Точки_продаж\time_live1 01-22.04.csv'
with open(file_encoding, 'rb') as difine_code:
    result = charset_normalizer.detect(difine_code.read(10000))
print(f'Наиболее вероятная кодировка: {result}')

def concat_all_csv_files(path):
    os.chdir(path)
    
    # Получаем список всех CSV файлов в папке
    csv_files = glob.glob('*.csv')
    
    # Сортируем файлы по дате изменения (от новых к старым)
    csv_files.sort(key=os.path.getmtime, reverse=True)
    
    combined = pd.DataFrame()
    
    try:
        for csv_file in csv_files:
            # Читаем CSV файл
            data = pd.read_csv(csv_file, skiprows=6, encoding='windows-1251', sep=';', parse_dates=True, on_bad_lines='warn',low_memory=False)  
            
            # Добавляем колонку с именем файла
            data['source_file'] = csv_file
            
            # Объединяем с общим DataFrame
            combined = pd.concat([combined, data], ignore_index=True)
        
        return combined
    
    except Exception as e:
        print(f"Произошла ошибка: {e}")
        return None

table = concat_all_csv_files(path)

table['Дата'] = pd.to_datetime(table['Дата'], format='%d.%m.%Y') 

table['Месяц'] = table['Дата'].dt.month

Наиболее вероятная кодировка: {'encoding': 'windows-1251', 'language': 'Russian', 'confidence': 1.0}


### Следующий шаг -формирование отчёта по выдаче клтенту товара за 15 минут
Первый шаг. ['От101до57ф', 'От101до30ф', 'От101до60ф'] - расчёты по времени в минутах, если ни в одном из столбцов нет времени, то ставим нет данных,  
нет данных, убираем, они нам не нужны

In [2]:
table['ЭО'] = table['ЭО'].astype(str)

columns_with_time = ['От101до57ф', 'От101до30ф', 'От101до60ф']
table['статус'] = table[columns_with_time].isnull().sum(axis=1)
table['статус'] = table['статус'].apply(lambda x: 'нет данных' if x == 3 else 'данные есть')
table = table[table['статус'] != 'нет данных']
mini_table = table[['ЭО', 'Дата', 'Время', 'От101до57ф', 'От101до30ф', 'От101до60ф']].sort_values('ЭО').iloc[:10]


### Может ли дублироваться время по: От101до57ф, От101до30ф, От101до60ф?

Если по итогу получем среднее - 1 и стандартное - 0, значит, дублей по значениям нет.
Можно упростить расчёт времени

In [3]:
def count_notna(row):
    result = pd.notna(row['От101до57ф']) + pd.notna(row['От101до30ф']) + pd.notna(row['От101до60ф']) 
    return result
    
mini_table['кол-во значений'] = mini_table.apply(count_notna, axis=1)
avg = mini_table['кол-во значений'].mean()
std = mini_table['кол-во значений'].std()
print(f'среднее: {avg}, стандартное: {std}')

среднее: 1.0, стандартное: 0.0


### Расчёт рабочего времени
по трём колонкам с временем заполняем пустые значения нулями -> складываем столбцы -> получаем рабочее время  
если что складываем по От101до57ф, От101до30ф, От101до60ф

In [4]:

table[columns_with_time] = table[columns_with_time].fillna(0) 
table['Рабочее время'] = table[columns_with_time].sum(axis=1)

table['статус'].value_counts()

статус
данные есть    236062
Name: count, dtype: int64

### Считаем кол-во строк/объём в одном комплекте

Несколько заказов на одного клиента с одной Электронной очередью = комплект

Часто встречаются ситуации, когда один человек делает несколько заказов/либо происходит дробление менеджером.  
Такие случаи мы *считаем*.  

ниже считаем объём и кол-во строк на один комплект

In [5]:
table['строк на один комплект'] = table.groupby(['Код к/аг', 'ЭО'])['КолТовСтр'].transform('sum').fillna(0).astype(int)

table['объём комплекта суммарный'] = table.groupby(['Код к/аг', 'ЭО'])['ОбъемДок(м3)'].transform('sum').fillna(0).astype(float)

table[['Номер', 'ЭО', 'Дата', 'Время', 'От101до57ф', 'От101до30ф', 'От101до60ф', 'КолТовСтр',
       'объём комплекта суммарный', 'строк на один комплект', 'Рабочее время']].sort_values('ЭО', ascending=False).iloc[:10]

,Номер,ЭО,Дата,Время,От101до57ф,От101до30ф,От101до60ф,КолТовСтр,объём комплекта суммарный,строк на один комплект,Рабочее время
3708,401/1173849,ЯЯ1838,2025-04-11,17:38,5.0,0.0,0.0,5.0,0.009844,5,5.0
1388,429/6227539055,ЯЯ1825,2025-04-11,17:25,3.0,0.0,0.0,1.0,0.005000,1,3.0
1373,429/10235472,ЯЯ1820,2025-04-11,17:20,2.0,0.0,0.0,1.0,0.028324,1,2.0
2497,999/7335398,ЯЯ1746,2025-04-01,16:46,2.0,0.0,0.0,1.0,0.001014,1,2.0
4016,401/11916113,ЯЯ1740,2025-04-14,16:40,0.0,0.0,7.0,1.0,0.000152,1,7.0
827,401/11915989-2,ЯЯ1739,2025-04-08,16:39,6.0,0.0,0.0,4.0,0.006302,4,6.0
3301,401/11916011,ЯЯ1731,2025-04-08,16:31,0.0,0.0,3.0,1.0,0.000500,1,3.0
975,401/11916030,ЯЯ1730,2025-04-09,16:30,0.0,3.0,0.0,1.0,0.000022,1,3.0
4015,401/11916112,ЯЯ1720,2025-04-14,16:20,0.0,0.0,5.0,1.0,0.001859,1,5.0
2511,999/7379563,ЯЯ1711,2025-04-01,16:11,2.0,0.0,0.0,1.0,0.001150,1,2.0


### Удаление дублирующихся заказов

Удаляем дублирующиеся заказы внутри комплектов, при это по времени оставляем - с наибольшим. Чтобы рассчитать время на одного клиента

In [6]:
table.sort_values('Рабочее время', ascending=False)

table_without_dubl = table.drop_duplicates(['Код к/аг', 'ЭО'], keep='first')

print(f'Значений до удаления дублей: {len(table)}, после удаления: {len(table_without_dubl)}')


Значений до удаления дублей: 236062, после удаления: 166817


#### Распределние времени и заказа
соответсвующе разбиваем время на диапазончики  
также, разбиваем заказ на "Маленький заказ" и "более 12 позиций или 1М3"  
думаю, названия говорят сами за себя

In [7]:

def time_distribution(row):
    if isinstance(row['Рабочее время'], str):
        if row['Рабочее время'] == "нет данных":
            return row['Рабочее время']
    elif row['Рабочее время'] <= 15:
        return "до 15"
    elif row['Рабочее время'] <= 30 and row['Рабочее время'] > 15:
        return "16-30"
    elif row['Рабочее время'] <= 45 and row['Рабочее время'] > 30:
        return "31-45"
    elif row['Рабочее время'] <= 60 and row['Рабочее время'] > 45:
        return "46-60"
    elif row['Рабочее время'] > 60:
        return "Свыше часа"


def size_of_order(row):
    if (row['ОбъемДок(м3)'] == "") and pd.isnull(row['КолТовСтр']) or pd.isnull(row['ОбъемДок(м3)']) and pd.isnull(row['КолТовСтр']):
        return "нет данных"
    else:
        try:
            volume = float(row['объём комплекта суммарный'])
            goods = int(row['строк на один комплект'])
        except ValueError:
            volume = 0
            goods = 0
        
        if volume > 1 or goods > 12:
            return "более 12 строк или 1М3"
        else:
            return "Маленький заказ"

table_without_dubl['Распределение времени'] = table_without_dubl.apply(time_distribution, axis=1)
table_without_dubl['Размер заказа'] = table_without_dubl.apply(size_of_order, axis=1)

table_without_dubl['Размер заказа'].value_counts()


C:\Users\Home\AppData\Local\Temp\ipykernel_34336\1794854112.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_without_dubl['Распределение времени'] = table_without_dubl.apply(time_distribution, axis=1)
C:\Users\Home\AppData\Local\Temp\ipykernel_34336\1794854112.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_without_dubl['Размер заказа'] = table_without_dubl.apply(size_of_order, axis=1)


Размер заказа
Маленький заказ           153765
более 12 строк или 1М3     13052
Name: count, dtype: int64

#### построение сводной таблицы  
Предварителньо здесь можно ознакомиться с результамтами, можно выбрать необходимый месяц

In [11]:
# table_current_m = table_current_m[table_current_m['Распределение времени'] != 'не выдан']
pivot_table_office = table_without_dubl.pivot_table( values='Номер',
    index=None,
    columns='Распределение времени',
    aggfunc='count')

pivot_table_office['Всего'] = pivot_table_office[['16-30', '31-45', '46-60', 'Свыше часа', 'до 15']].sum(axis=1)
pivot_table_office['% До 15'] = pivot_table_office['до 15'] / pivot_table_office['Всего'] 
pivot_table_office['% 16-30'] = pivot_table_office['16-30'] / pivot_table_office['Всего'] 
pivot_table_office['% 31-45'] = pivot_table_office['31-45'] / pivot_table_office['Всего'] 
pivot_table_office['% 46-60'] = pivot_table_office['46-60'] / pivot_table_office['Всего'] 
pivot_table_office['% Свыше часа'] = pivot_table_office['Свыше часа'] / pivot_table_office['Всего'] 

pivot_table_office

Распределение времени,16-30,31-45,46-60,Свыше часа,до 15,Всего,% До 15,% 16-30,% 31-45,% 46-60,% Свыше часа
Номер,5632,1004,356,1893,157932,166817,0.946738,0.033762,0.006019,0.002134,0.011348


#### Выгружаем по итогу обработанные данные  


In [ ]:
archive_path_1 = f"C:\\Users\\Home\\Desktop\\Data\\Точки_продаж\\данные своевременность по ОП, ВТЦ, СВК {today}.xlsx"
with pd.ExcelWriter(archive_path_1, engine='openpyxl') as writer:
    table_without_dubl.to_excel(writer, sheet_name='отчёт', index=False)